# Introduction

In this post, I try to write a simple homegrown version of `std::unique_ptr<T>`. This post is partly inspired by the fantastic book [C++ Memory Management](https://www.amazon.com/Memory-Management-leaner-memory-management-techniques/dp/1805129805) by *Patrice Roy*. Tghe toy examples in this book are very instructive and I highly reckon you order a copy. Our goal is just to build intuition for the kind of code required to write such a type, and not to try and replace the standard library facilities.

The `std::unique_ptr<T>` smart pointer type models unqiue(sole) ownership of the resource semantics.

```cpp
struct X{};

std::unique_ptr<X> p1 = std::make_unique<X>();  
//std::unique_ptr<X> p2(p1);      // Error when calling copy constructor, 
                                  // p1 is the exclusive owner
```

`std::unique_ptr` enforces exclusive ownership using the fact, that it is not *copy-constructible* or *copy-assignable*. Note however, that it doesn't prevent you from writing deliberately hostile code. The below code is compiles perfectly well and is valid C++.

```cpp
int* p = new int(10);

std::unique_ptr<int> p1(p);  
std::unique_ptr<int> p2(p);      
```

The copy constructor and the copy assignment operator of `std::unique_ptr<T>` are marked `delete`.  It is however, *move constructible* and *move-assignable*.

# Basic functionalities to expect out of `std::unique_ptr<T>`

I skimmed through the documentation for `std::unique_ptr` on cppreference.com. A basic implementation of `unique_ptr` in less than 200 lines of code should pass the following unit tests:


```{cpp}
{% include "https://raw.githubusercontent.com/quasar-chunawala/interview_data_structures/refs/heads/main/tests/unique_ptr_test/unique_ptr_test.cpp" %}
```


# A custom implementation

```cpp
// UniquePointer.cpp : This file contains the 'main' function. Program execution begins and ends there.

#include <iostream>
#include <memory>
#include <cassert>
#include <utility>
#include <vector>
#include <optional>


namespace dev {
    // Implement unique_ptr here
    template<typename T>
    class unique_ptr {
    public:
        // Default c'tor 
        unique_ptr() : ptr{ nullptr } {}

        // Copy c'tor should be deleted to enforce the concept that this is 
        // an owning pointer
        unique_ptr(const unique_ptr& u) = delete;

        // Copy assignment should also be deleted to enforce the ownership of the 
        // managed object
        unique_ptr& operator=(const unique_ptr&) = delete;

        // Move constructor
        unique_ptr(unique_ptr&& other) : ptr{ nullptr }
        {
            std::swap(ptr, other.ptr);
        }

        // Move assignment operator
        unique_ptr& operator=(unique_ptr&& other) {
            if (ptr == other)
                return (*this);

            delete_underlying_ptr();
            ptr = std::exchange(other.ptr, nullptr);
            return (*this);
        }

        // Parameterized construtor
        unique_ptr(T* p)
            : ptr{ p }
        {
        }

        // Overload deferencing operator *
        T& operator*() {
            return (*ptr);
        }

        // Overload deferencing operator *
        T& operator*() const {
            return (*ptr);
        }

        // get the raw pointer
        T* get() const
        {
            return ptr;
        }

        /* Reset the unique_ptr */
        void reset(T* other) {
            if (ptr == other)
                return;

            delete_underlying_ptr();
            std::swap(ptr, other);
        }

        /* Release unique_ptr ownership */
        T* release() {
            return std::exchange(ptr, nullptr);
        }

        /* Destructor */
        ~unique_ptr() {
            delete_underlying_ptr();
        }

        /* swap - Exchange the contents of ptr1 and ptr2 member-by-member */
        friend void swap(dev::unique_ptr<T>& ptr1, dev::unique_ptr<T>& ptr2) noexcept
        {
            //Re-wire the raw pointers
            std::swap(ptr1.ptr, ptr2.ptr);
        }

        /* operator bool */
        constexpr operator bool() {
            return (ptr != nullptr);
        }

        /* Member access operator - return the underlying pointer */
        T* operator->() {
            return ptr;
        }

        /* Helper function to invoke delete on the underlying raw pointer */
        void delete_underlying_ptr() {
            if (ptr != nullptr) {
                delete ptr;
                ptr = nullptr;
            }
        }
    private:
        T* ptr;
    };

    template<typename T, typename... Args>
    dev::unique_ptr<T> make_unique(Args&&... args) {
        return dev::unique_ptr(new T(std::forward<Args>(args)...));
    }

    /* Non-member functions */
//Overload operator==
    template<typename T1, typename T2>
    bool operator==(const dev::unique_ptr<T1>& lhs, const dev::unique_ptr<T2>& rhs) {
        return lhs.get() == rhs.get();
    }

    /*template<typename T1> */
    template<typename T1>
    bool operator==(const dev::unique_ptr<T1>& lhs, std::nullptr_t rhs) {
        return lhs.get() == nullptr;
    }

    template<typename T1>
    bool operator==(std::nullptr_t lhs, const dev::unique_ptr<T1>& rhs) {
        return rhs.get() == nullptr;
    }

    //Overload operator!=
    template<typename T1, typename T2>
    bool operator!=(const dev::unique_ptr<T1>& lhs, const dev::unique_ptr<T2>& rhs) {
        return !(lhs == rhs);
    }

    template<typename T1>
    bool operator!=(const dev::unique_ptr<T1>& lhs, std::nullopt_t& rhs) {
        return !(lhs == rhs);
    }

    template<typename T1>
    bool operator!=(std::nullopt_t& lhs, const dev::unique_ptr<T1>& rhs) {
        return !(lhs == rhs);
    }
}
```

[Compiler Explorer](https://godbolt.org/z/vMzcYzhGe)

## References {.appendix}

- *[C++ Memory Management](https://www.amazon.com/Memory-Management-leaner-memory-management-techniques/dp/1805129805) by *Patrice Roy*.